I found the code for the perlin noise online, unfortunately its differential is not available, and I need it to calculate normals. The numerical differential formula starts glitching for large range. Now I am not emotinally stable enough to sit through having to calculate it by hand, so Imma just make a computation graph and use that instead.

In [6]:
import jax
from jax import numpy as jnp
from jax import grad

In [7]:
hehe = jnp.array([1,2,3,4])

In [17]:
jnp.sum(hehe * hehe)

DeviceArray(30, dtype=int32)

In [57]:
    def fade(t): 
        return t*t*t*(t*(t*6.0-15.0)+10.0)
    def permute(x):
        return jnp.mod(((x*34.0)+1.0)*x, 289.0)
    def dot(x,y):
        return jnp.sum(x * y)

    def cnoise(x,y):
        Pf, Pi = jnp.modf(jnp.array([x,y,x,y]))
        Pi += jnp.array([0.0, 0.0, 1.0, 1.0]) 
        Pf -= jnp.array([0.0, 0.0, 1.0, 1.0]) 
        Pi = jnp.mod(Pi , 289.0)  
        ix = jnp.array([Pi[0],Pi[2],Pi[0],Pi[2]]) 
        iy = jnp.array([Pi[1], Pi[1], Pi[3], Pi[3]]) 
        fx = jnp.array([Pf[0], Pf[2], Pf[0], Pf[2]]) 
        fy = jnp.array([Pf[1], Pf[1], Pf[3], Pf[3]]) 
        i = permute(permute(ix) + iy) 
        gx = 2.0 * jnp.modf(i * 0.0243902439)[0] - 1.0
        gy = abs(gx) - 0.5
        tx = jnp.floor(gx + 0.5) 
        gx = gx - tx 
        g00 = jnp.array([gx[0],gy[0]]) 
        g10 = jnp.array([gx[1],gy[1]]) 
        g01 = jnp.array([gx[2],gy[2]]) 
        g11 = jnp.array([gx[3],gy[3]]) 
        norm = 1.79284291400159 - 0.85373472095314 * jnp.array([dot(g00, g00), dot(g01, g01), dot(g10, g10), dot(g11, g11)]) 
        g00 *= norm[0] 
        g01 *= norm[1] 
        g10 *= norm[2] 
        g11 *= norm[3] 
        n00 = dot(g00, jnp.array([fx[0], fy[0]])) 
        n10 = dot(g10, jnp.array([fx[1], fy[1]])) 
        n01 = dot(g01, jnp.array([fx[2], fy[2]])) 
        n11 = dot(g11, jnp.array([fx[3], fy[3]])) 
        fade_xy = fade(jnp.array([Pf[0], Pf[1]])) 
        n_x = jnp.array([n00, n01]) * (1 - fade_xy[0]) +  jnp.array([n10, n11]) * fade_xy[0] 
        n_xy = n_x[0] * (1 - fade_xy[1]) + n_x[1] * fade_xy[1] 
        return 2.3 * n_xy 


In [58]:
for i in range(3):
    for j in range(0, 10):
        print(i,j, cnoise(i/10,j/10))

2 0 -0.06484855
2 1 0.093980774
2 2 0.20321386
2 3 0.23280133
2 4 0.17662016
2 5 0.051779356
2 6 -0.1059663
2 7 -0.25069636
2 8 -0.33869827
2 9 -0.34473214
